# Comparison

In this notebook we compare composite maps and optimal projection patterns

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


import pandas as pd
import xarray as xr
import cartopy.crs as ccrs

from tqdm.notebook import tqdm

import sys
sys.path.append('../../../Climate-Learning/')

import general_purpose.utilities as ut
import general_purpose.cartopy_plots as cplt
import general_purpose.uplotlib as uplt
import general_purpose.tables as tbl

# log to stdout
import logging
logging.getLogger().level = logging.INFO
logging.getLogger().handlers = [logging.StreamHandler(sys.stdout)]
ut.indentation_sep = '  '

HOME = '../../'

In [ ]:
lon = np.load('../../lon.npy')
lat = np.load('../../lat.npy')
LON, LAT = np.meshgrid(lon, lat)

## Figure 11

In [ ]:
mask = np.ones((22,128,1), dtype=bool)
reshaper = ut.Reshaper(mask)
reshaper.surviving_coords

### PLASIM 80y Z composites

In [ ]:
ds = xr.open_dataset('../../PLASIM/composites/composite_maps_T1_percent5_y80-Z.nc')
ds

In [ ]:
ga = ds.gaussian_composite.data.reshape(4,22,128,3)
ga.shape

In [ ]:
ga_Z = ga[...,1:2]
ga_Z.shape

In [ ]:
## area weights

coslat = np.cos(lat*np.pi/180)
aw = (np.ones(mask.shape).T * coslat).T
aw *= mask
aw /= np.sum(aw)
aw.shape

In [ ]:
awZ = aw[...,1:2]
awZ /= np.sum(awZ)
awZ.shape

In [ ]:
norm = l2(ga_Z*np.sqrt(awZ), axis =(1,2,3))
ga_Z = (ga_Z.T / norm).T
ga_Z.shape

In [ ]:
# This plots the first column of Fig.11
for i,tau in enumerate(ds.tau.data):
    axs = cplt.mfp(LON, LAT, ga_Z[i], one_fig_layout=110, figsize=(5,5), fig_num=8+i, colorbar='individual', titles=f'|C|={np.round(norm[i],2)}',mx=4,put_colorbar=False,)
    fig = axs[0].get_figure()
    # fig.savefig(f'{HOME}PLASIM-y80_C_T1_tau{tau}.pdf')

### ERA5 composites

In [ ]:
root_folder = '../ERA5/y83'

Model = 'ERA5'

# Northern Hemisphere
sector = ''
mask = 1

In [ ]:
import general_purpose.cartopy_plots as cplt

lon = np.load(f'{root_folder}/lon.npy')
lon = cplt.monotonize_longitude(lon)
lat = np.load(f'{root_folder}/lat.npy')
LON, LAT = np.meshgrid(lon,lat)

def retrieve_maps(T, tau, percent):
    folder = f'{root_folder}/T{T}/tau{tau}/percent{percent}'
    comp = np.load(f'{folder}/X_comp.npy')
    comp_std = np.load(f'{folder}/X_comp_std.npy')
    comp_ga = np.load(f'{folder}/X_comp_GA.npy')
    
    return comp, comp_ga, comp_std

_,_,Xcstd = retrieve_maps(14,0,5)
reshaper = ut.Reshaper(Xcstd != 0)
print(reshaper.surviving_coords)

In [ ]:
T = 1
taus = [0,2,4,6]
percent = 5

In [ ]:
norms = np.round(ds.total_aw_norm.sel(T=T, percent = percent, tau = taus).data,2)

In [ ]:
for i,tau in enumerate(taus):
    comp, comp_ga, comp_std = retrieve_maps(T, tau, percent)
    axs = cplt.mfp(LON, LAT, comp_ga/norms[i], one_fig_layout=110, figsize=(5,5), fig_num=8+i, colorbar='individual', titles=f'|C|={norms[i]}',mx=4,put_colorbar=False,)
    fig = axs[0].get_figure()
    # fig.savefig(f'{HOME}ERA5_C_T{T}_tau{tau}.pdf')

### PLASIM 80y Z projection patterns

In [ ]:
Ms = xr.open_dataarray('../../PLASIM/committor/projection_patterns_T1_y80_epsilonbest_fold0-Z.nc')
Ms

In [ ]:
ss = xr.open_dataarray('../../PLASIM/committor/Skill-GA_percent5_y80_epsilonbest-Z.nc').sel(T=Ms['T'], tau=Ms['tau'])
ss

In [ ]:
_proj = []
titles = []
nfig = len(ss['tau'].data)
for tau in ss['tau'].data:
    sss = ss.sel(tau=tau)
    eps = int(np.log10(sss['reg_c'].data.item()))
    proj = reshaper.inv_reshape(Ms.sel(tau=tau).data.squeeze())
    S = uplt.xr_avg(sss, 'fold').data.squeeze().item()
    _proj.append(proj)
    
    # pretit = fr'$\tau={tau}$' + '\n'
    pretit = ''
    
    titles.append(fr'{pretit}$\epsilon_\mathrm{{best}}=10^{{{eps}}}, \mathcal{{S}} = {S:L}$')
    
_proj = np.concatenate(_proj, axis=-1)
_ = cplt.multiple_field_plot(LON, LAT, _proj, titles=titles, colorbar='shared', mode='pcolormesh', draw_gridlines=False,
                             put_colorbar=False,
                             # one_fig_layout=100+10*nfig, figsize=(nfig*5,5),
                             figsize=(5,5),
                             mx=4)

In [ ]:
for i,tau in enumerate(ss['tau'].data):
    fig = _[i].get_figure()
    fig.savefig(f'{HOME}/PLASIM-y80_M_T1_tau{tau}.pdf')

### ERA5 projection patterns

In [ ]:
Ms = xr.open_dataarray('../committor/projection_patterns_T1_epsilonbest_fold0.nc')
Ms

In [ ]:
ss = xr.open_dataarray('../committor/Skill-GA_percent5_epsilonbest.nc').sel(T=Ms['T'], tau=Ms['tau'])
ss

In [ ]:
_proj = []
titles = []
nfig = len(ss['tau'].data)
for tau in ss['tau'].data:
    sss = ss.sel(tau=tau)
    eps = int(np.log10(sss['reg_c'].data.item()))
    proj = reshaper.inv_reshape(Ms.sel(tau=tau).data.squeeze())
    S = uplt.xr_avg(sss, 'fold').data.squeeze().item()
    _proj.append(proj)
    
    pretit = fr'$\tau={tau}$' + '\n'
    pretit = ''
    
    titles.append(fr'{pretit}$\epsilon_\mathrm{{best}}=10^{{{eps}}}, \mathcal{{S}} = {S:L}$')
    
_proj = np.concatenate(_proj, axis=-1)
_ = cplt.multiple_field_plot(LON, LAT, _proj, titles=titles, colorbar='shared', mode='pcolormesh', draw_gridlines=False,
                             put_colorbar=False,
                             # one_fig_layout=100+10*nfig, figsize=(nfig*5,5),
                             figsize=(5,5),
                             mx=4)

In [ ]:
for i,tau in enumerate(ss['tau'].data):
    fig = _[i].get_figure()
    fig.savefig(f'{HOME}/ERA5_M_T1_tau{tau}.pdf')

## Figure 1

In [ ]:
root_folder = 'y83'

Model = 'ERA5'

# Northern Hemisphere
sector = ''
mask = 1

In [ ]:
lon = np.load(f'{root_folder}/lon.npy')
lon = cplt.monotonize_longitude(lon)
lat = np.load(f'{root_folder}/lat.npy')
LON, LAT = np.meshgrid(lon,lat)

def retrieve_maps(T, tau, percent):
    folder = f'{root_folder}/T{T}/tau{tau}/percent{percent}'
    comp = np.load(f'{folder}/X_comp.npy')
    comp_std = np.load(f'{folder}/X_comp_std.npy')
    comp_ga = np.load(f'{folder}/X_comp_GA.npy')
    
    return comp, comp_ga, comp_std

_,_,Xcstd = retrieve_maps(14,0,5)
reshaper = ut.Reshaper(Xcstd != 0)
print(reshaper.surviving_coords)

In [ ]:
T = 14
tau=0

for percent in [3,5]:
    compERA, comp_gaERA, comp_stdERA = retrieve_maps(T,tau,percent)
    compCESM = xr.open_dataset('composites_CESM.nc').empirical_composite.sel(percent = percent).data.reshape([22,128,1])
    
    if percent == 5:
        compPLASIM = xr.open_dataset('../../PLASIM/composites/composite_maps_percent5_y8000.nc').empirical_composite.sel(T=T,tau=tau).data.reshape(22,128,3)[:,:,1].reshape(22,128,1)
    else:
        compPLASIM= np.load('../../PLASIM/composites/composite_map_T14_tau0_percent3_y8000-Z.npy')
    
    
    composites = np.concatenate([compERA,compPLASIM,compCESM],axis=2)
    
    axs = cplt.mfp(LON,LAT,composites,projections=ccrs.Orthographic(central_latitude=90),
                   extents=None,one_fig_layout=130,colorbar='shared', put_colorbar= False, figsize=(13,5),mx =1.05,fig_num=8+np.random.randint(0,50),titles=None)
    
    fig = axs[0].get_figure()
    fig.savefig(f'{HOME}composite_{percent}.pdf')
    

In [ ]:
plt.close(1)
fig,ax = plt.subplots(figsize=(0.5,10), num=1)
plt.colorbar(axs[0], cax=ax, extend='both')
fig.tight_layout()
plt.savefig(f'{HOME}colorbar-composite.pdf',bbox_inches = 'tight')